# Data Cleaning and analysation of the German GHG quota

The last notebook deals with data cleaning and analysis of the scraped data.

The analysis is divided into providers that did not change their prices and those that did.

If you have a suggestion for improvement, please let me know.

In [59]:
import pandas as pd
import numpy as np
import plotly.express as px
import sqlalchemy
import ast
import json
import warnings
warnings.filterwarnings('ignore')
import datetime
import os

In [60]:
actual_month = datetime.datetime.today().strftime('%B')

In [61]:
if not os.path.exists("Monatsberichte"):
    os.mkdir("Monatsberichte")

### Read Database to create DataFrame

In [62]:
engine = sqlalchemy.create_engine('sqlite:///verivox_thg.db')

with engine.connect() as conn:
    df_raw = pd.read_sql_table('thg', conn) 

### Format date column, clean DataFrame and delete unneeded columns and flex entries

In [63]:
df_raw['Datum'] = pd.to_datetime(df_raw['Datum'])

In [64]:
import ast

def extract_key_metrix(raw): # function to extract all needed values from saved raw data set
    data_dict = ast.literal_eval(raw)
    js_dump = json.dumps(data_dict)
    js_load = json.loads(js_dump)
    js_norm = pd.json_normalize(js_load)['keyMetrics'][0]
    
    value_list = []
    
    for i in js_norm:
        i_out = ast.literal_eval(str(i))
        # remove '<b>THG Preismodell: Fix-Prämie</b>' for better view because it's part of every entry
        # if you run the code without "Auszahlung" condition you can see a few lines below that this is the main difference
        # for a better groupby we set the condition here
        if i_out['value'] != '<b>THG Preismodell: Fix-Prämie</b>' and "Auszahlung " in i_out['value']:
            value_list.append(i_out['value'])

            
    feature = str(value_list)
    
    feature = feature.replace('[', '')
    feature = feature.replace(']', '')
    feature = feature.replace("'", '')       
    
    return feature

In [65]:
df_raw['Eigenschaft'] = df_raw['raw_data'].apply(extract_key_metrix)

In [66]:
df_raw = df_raw[['Datum', 'Provider', 'Preis', 'Bezahlmodel', 'Eigenschaft']]

In [67]:
df_raw.drop_duplicates(inplace=True, ignore_index=True)

In [68]:
df_raw.drop(df_raw[df_raw['Bezahlmodel']=='flex'].index, axis=0, inplace=True)

In [69]:
df_raw.reset_index(drop=True, inplace=True)

### Clean data from daily to weekly

In [70]:
df_raw['KW'] = df_raw['Datum'].apply(lambda x: x.isocalendar().week)

In [71]:
df_raw.drop(df_raw[df_raw['KW']==52].index, inplace=True)

In [72]:
df_raw.drop_duplicates(subset=['KW', 'Eigenschaft','Provider'], keep='last', inplace=True)

In [73]:
df_raw = df_raw[df_raw['Datum'].dt.year==2024]

In [74]:
# df_raw = df_raw[df_raw['KW']>=8]

# Unchanged providers

In [75]:
unchanged_provider = (df_raw.groupby(by=["Provider", "Bezahlmodel", "Preis", "Eigenschaft"]).size().reset_index().groupby(by=['Provider', 'Bezahlmodel', 'Eigenschaft']).size()==1).to_frame().reset_index()

In [76]:
unchanged_provider = unchanged_provider[unchanged_provider[0]==True].drop(0, axis=1)

In [77]:
df_unchanged_prices = df_raw.groupby(by=["Provider", "Bezahlmodel", "Preis", 'Eigenschaft']).size().to_frame().reset_index().drop(0, axis=1)

In [78]:
df_unchanged = pd.merge(unchanged_provider, df_unchanged_prices, how='left', left_on=['Provider', 'Bezahlmodel', 'Eigenschaft'], right_on=['Provider', 'Bezahlmodel', 'Eigenschaft'])

In [79]:
df_unchanged.sort_values(by=['Preis'], ascending=False, inplace=True)

In [80]:
df_unchanged = pd.merge(
    df_unchanged,
    df_raw,
    how='inner',
    on=['Provider','Preis','Bezahlmodel','Eigenschaft']
).drop('Datum', axis=1)

In [81]:
df_unchanged = df_unchanged.sort_values(by=['Provider', 'Eigenschaft','KW'], ascending=True).drop_duplicates(subset=['Provider', 'Bezahlmodel', 'Eigenschaft', 'Preis'], keep='last').sort_values(by=['Preis'], ascending=False)

In [82]:
df_unchanged.to_excel(os.path.join("Monatsberichte", f"Unveränderte Angebote_Stand {actual_month}.xlsx"), index=None)

# Changed providers

In [83]:
df_changed = pd.merge(df_raw, df_unchanged, indicator=True, how='outer', on=['Provider', 'Bezahlmodel', 'Eigenschaft', 'Preis']).query('_merge=="left_only"').drop(['_merge', 'KW_y'], axis=1).rename({"KW_x":"KW"}, axis=1)

In [84]:
df_changed.sort_values(by=['Datum'], inplace=True)

df_changed = df_changed.reset_index(drop=True)

In [85]:
def define_fig_name(row):
    model = row['Bezahlmodel']
    label = row['Eigenschaft']
    prov = row['Provider']
    
    return f"{prov}_{model}_{label}"

In [86]:
df_changed['Neuer Name'] = df_changed.apply(define_fig_name, axis=1)

In [87]:
fig = px.line(df_changed,
       y='Preis',
       x='KW',
       color='Neuer Name')

In [88]:
fig.update_layout(xaxis=dict(tickmode='linear', dtick=1), 
                 legend_title_text=None)

In [89]:
fig.write_html(os.path.join("Monatsberichte", f"Angebote Verivox mit Veränderungen_Stand {actual_month}.html"))